In [1]:
# 한글 폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

print("한글 폰트 설치 완료!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,354 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126374 files and dire

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [4]:
# 3. 최신 변경사항 가져오기
!git pull origin main

From https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
# 3. 현재 상태 확인
!git status

Refresh index: 100% (13/13), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/create_dataset.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data_preprocess/Untitled0.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


# 데이터 전처리 코드

## 1단계: 데이터 품질 검증 및 필터링

### 초기 설정

In [19]:
import os
import json
import glob
from PIL import Image
import pandas as pd
from collections import defaultdict, Counter

### a.🔍 이미지 품질검사

In [6]:
def check_image_quality(data_path="./data"):
    """이미지 품질 검사"""
    print("🔍 이미지 품질 검사 시작...")

    # 결과 저장용
    results = {
        'total_images': 0,
        'valid_images': 0,
        'corrupted_images': [],
        'small_images': [],
        'invalid_format': [],
        'image_stats': []
    }

    # 훈련 이미지 검사
    train_images = glob.glob(os.path.join(data_path, "train_images", "*.png"))
    test_images = glob.glob(os.path.join(data_path, "test_images", "*.png"))
    all_images = train_images + test_images

    results['total_images'] = len(all_images)
    print(f"📊 총 이미지 수: {len(all_images)}개")

    for i, img_path in enumerate(all_images):
        try:
            # 이미지 열기 시도
            with Image.open(img_path) as img:
                width, height = img.size
                mode = img.mode

                # 최소 해상도 검사 (100x100)
                if width < 100 or height < 100:
                    results['small_images'].append({
                        'path': img_path,
                        'size': (width, height)
                    })
                    continue

                # RGB 채널 확인
                if mode not in ['RGB', 'L']:  # L은 grayscale
                    results['invalid_format'].append({
                        'path': img_path,
                        'mode': mode
                    })
                    continue

                # 정상 이미지
                results['valid_images'] += 1
                results['image_stats'].append({
                    'path': img_path,
                    'width': width,
                    'height': height,
                    'mode': mode,
                    'size_mb': os.path.getsize(img_path) / (1024*1024)
                })

        except Exception as e:
            # 손상된 이미지
            results['corrupted_images'].append({
                'path': img_path,
                'error': str(e)
            })

        # 진행률 출력
        if (i + 1) % 100 == 0:
            print(f"  진행률: {i + 1}/{len(all_images)}")

    # 결과 요약
    print(f"\n📋 이미지 품질 검사 결과:")
    print(f"  ✅ 정상 이미지: {results['valid_images']}개")
    print(f"  ❌ 손상된 이미지: {len(results['corrupted_images'])}개")
    print(f"  ⚠️ 너무 작은 이미지: {len(results['small_images'])}개")
    print(f"  🔄 잘못된 형식: {len(results['invalid_format'])}개")

    return results

### b. 🔍 어노테이션 무결성 검사

In [10]:
def check_annotation_integrity(data_path="./data"):
    """어노테이션 무결성 검사"""
    print("\n🔍 어노테이션 무결성 검사 시작...")

    results = {
        'total_annotations': 0,
        'valid_annotations': 0,
        'corrupted_json': [],
        'missing_pairs': [],
        'invalid_bbox': [],
        'annotation_stats': []
    }

    # 훈련 이미지와 어노테이션 매칭 확인
    train_images = glob.glob(os.path.join(data_path, "train_images", "*.png"))

    for img_path in train_images:
        img_name = os.path.basename(img_path).replace('.png', '')

        # 해당하는 JSON 파일 찾기
        json_files = glob.glob(os.path.join(data_path, "train_annotations", "*", "*", f"{img_name}.json"))

        if not json_files:
            # 매칭되는 어노테이션이 없음
            results['missing_pairs'].append({
                'image': img_path,
                'reason': 'No matching JSON file'
            })
            continue

        json_path = json_files[0]
        results['total_annotations'] += 1

        try:
            # JSON 파일 읽기 시도
            with open(json_path, 'r', encoding='utf-8') as f:
                annotation_data = json.load(f)

            # 이미지 크기 가져오기 (바운딩 박스 검증용)
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size
            except:
                continue

            # 바운딩 박스 검증 (기본적인 검사)
            bbox_valid = True
            if 'annotations' in annotation_data:
                for ann in annotation_data['annotations']:
                    if 'bbox' in ann:
                        bbox = ann['bbox']
                        # bbox 형식 확인 [x, y, width, height] 또는 [x_min, y_min, x_max, y_max]
                        if len(bbox) == 4:
                            x, y, w, h = bbox
                            # 좌표가 이미지 범위 내에 있는지 확인
                            if x < 0 or y < 0 or x + w > img_width or y + h > img_height:
                                bbox_valid = False
                                break

            if not bbox_valid:
                results['invalid_bbox'].append({
                    'annotation': json_path,
                    'image_size': (img_width, img_height),
                    'reason': 'Bbox outside image boundaries'
                })
                continue

            # 정상 어노테이션
            results['valid_annotations'] += 1

            # 통계 정보 수집
            pill_codes = []
            if 'annotations' in annotation_data:
                for ann in annotation_data['annotations']:
                    if 'category_id' in ann:
                        pill_codes.append(ann['category_id'])

            results['annotation_stats'].append({
                'path': json_path,
                'image_path': img_path,
                'num_objects': len(annotation_data.get('annotations', [])),
                'pill_codes': pill_codes
            })

        except json.JSONDecodeError as e:
            # JSON 파싱 오류
            results['corrupted_json'].append({
                'path': json_path,
                'error': str(e)
            })
        except Exception as e:
            # 기타 오류
            results['corrupted_json'].append({
                'path': json_path,
                'error': str(e)
            })

    # 결과 요약
    print(f"📋 어노테이션 무결성 검사 결과:")
    print(f"  ✅ 정상 어노테이션: {results['valid_annotations']}개")
    print(f"  ❌ 손상된 JSON: {len(results['corrupted_json'])}개")
    print(f"  🔗 매칭되지 않는 쌍: {len(results['missing_pairs'])}개")
    print(f"  📐 잘못된 바운딩 박스: {len(results['invalid_bbox'])}개")

    return results

### c.📊 데이터셋 기본 통계 정보 생성

In [14]:
def generate_dataset_statistics(image_results, annotation_results):
    """데이터셋 기본 통계 정보 생성"""
    print("\n📊 데이터셋 통계 정보 생성...")

    # 이미지 통계
    if image_results['image_stats']:
        widths = [stat['width'] for stat in image_results['image_stats']]
        heights = [stat['height'] for stat in image_results['image_stats']]
        sizes_mb = [stat['size_mb'] for stat in image_results['image_stats']]

        print(f"\n📐 이미지 크기 통계:")
        print(f"  평균 크기: {sum(widths)/len(widths):.0f} x {sum(heights)/len(heights):.0f}")
        print(f"  최소 크기: {min(widths)} x {min(heights)}")
        print(f"  최대 크기: {max(widths)} x {max(heights)}")
        print(f"  평균 파일 크기: {sum(sizes_mb)/len(sizes_mb):.2f} MB")

    # 어노테이션 통계
    if annotation_results['annotation_stats']:
        total_objects = sum(stat['num_objects'] for stat in annotation_results['annotation_stats'])
        avg_objects_per_image = total_objects / len(annotation_results['annotation_stats'])

        print(f"\n🏷️ 어노테이션 통계:")
        print(f"  총 객체 수: {total_objects}개")
        print(f"  이미지당 평균 객체 수: {avg_objects_per_image:.1f}개")

        # 알약 코드 분포 (간단하게)
        all_codes = []
        for stat in annotation_results['annotation_stats']:
            all_codes.extend(stat['pill_codes'])

        if all_codes:
            code_counts = Counter(all_codes)
            print(f"  고유 알약 종류: {len(code_counts)}개")
            print(f"  가장 많은 알약 코드:")
            for code, count in code_counts.most_common(5):
                print(f"    {code}: {count}개")

### d. 📋 품질 검사 보고서 생성

In [15]:
def create_quality_report(image_results, annotation_results, output_path="./data"):
    """품질 검사 보고서 생성"""
    print(f"\n📋 품질 검사 보고서 생성 중...")

    report = {
        'summary': {
            'total_images': image_results['total_images'],
            'valid_images': image_results['valid_images'],
            'total_annotations': annotation_results['total_annotations'],
            'valid_annotations': annotation_results['valid_annotations'],
            'overall_quality_score': (
                (image_results['valid_images'] / max(image_results['total_images'], 1) * 0.5) +
                (annotation_results['valid_annotations'] / max(annotation_results['total_annotations'], 1) * 0.5)
            ) * 100
        },
        'issues': {
            'corrupted_images': len(image_results['corrupted_images']),
            'small_images': len(image_results['small_images']),
            'corrupted_json': len(annotation_results['corrupted_json']),
            'missing_pairs': len(annotation_results['missing_pairs']),
            'invalid_bbox': len(annotation_results['invalid_bbox'])
        }
    }

    # JSON 형태로 저장
    report_path = os.path.join(output_path, "quality_report.json")
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)

    print(f"✅ 보고서 저장 완료: {report_path}")
    print(f"\n🎯 전체 품질 점수: {report['summary']['overall_quality_score']:.1f}%")

    return report

### e. 🔍 발견된 문제들 상세 출력

In [16]:
def print_detailed_issues(image_results, annotation_results, max_display=5):
    """발견된 문제들 상세 출력"""
    print(f"\n🔍 발견된 문제들 (최대 {max_display}개씩 표시):")

    # 손상된 이미지
    if image_results['corrupted_images']:
        print(f"\n❌ 손상된 이미지 ({len(image_results['corrupted_images'])}개):")
        for i, issue in enumerate(image_results['corrupted_images'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     오류: {issue['error']}")

    # 너무 작은 이미지
    if image_results['small_images']:
        print(f"\n⚠️ 너무 작은 이미지 ({len(image_results['small_images'])}개):")
        for i, issue in enumerate(image_results['small_images'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     크기: {issue['size']}")

    # 손상된 JSON
    if annotation_results['corrupted_json']:
        print(f"\n❌ 손상된 어노테이션 ({len(annotation_results['corrupted_json'])}개):")
        for i, issue in enumerate(annotation_results['corrupted_json'][:max_display]):
            print(f"  {i+1}. {issue['path']}")
            print(f"     오류: {issue['error']}")

    # 매칭되지 않는 쌍
    if annotation_results['missing_pairs']:
        print(f"\n🔗 매칭되지 않는 이미지-어노테이션 쌍 ({len(annotation_results['missing_pairs'])}개):")
        for i, issue in enumerate(annotation_results['missing_pairs'][:max_display]):
            print(f"  {i+1}. {issue['image']}")

### f. 🚀 전체 품질 검사 실행

In [17]:
def run_quality_check(data_path="./data"):
    """전체 품질 검사 실행"""
    print("🚀 데이터 품질 검사 시작!")
    print("=" * 50)

    # 1. 이미지 품질 검사
    image_results = check_image_quality(data_path)

    # 2. 어노테이션 무결성 검사
    annotation_results = check_annotation_integrity(data_path)

    # 3. 통계 정보 생성
    generate_dataset_statistics(image_results, annotation_results)

    # 4. 품질 보고서 생성
    report = create_quality_report(image_results, annotation_results, data_path)

    # 5. 상세 문제 출력
    print_detailed_issues(image_results, annotation_results)

    print("\n" + "=" * 50)
    print("✅ 데이터 품질 검사 완료!")

    return image_results, annotation_results, report

## 메인 실행부

In [20]:
if __name__ == "__main__":
    # 코랩에서 실행
    print("📖 사용법:")
    print("run_quality_check(data_path)  # 전체 품질 검사 실행")
    print("run_quality_check(prototype_path)  # 소규모 데이터셋 검사")

    # 파일 경로들
    prototype_path = "./data/prototype_data"
    data_path = "./data"

    # 함수 실행
    # image_results, annotation_results, report = run_quality_check(data_path)    # 전체 데이터용
    image_results, annotation_results, report = run_quality_check(prototype_path)   # 소규모 데이터용

📖 사용법:
run_quality_check('./data')  # 전체 품질 검사 실행
run_quality_check('./small_data')  # 소규모 데이터셋 검사
🚀 데이터 품질 검사 시작!
🔍 이미지 품질 검사 시작...
📊 총 이미지 수: 75개

📋 이미지 품질 검사 결과:
  ✅ 정상 이미지: 75개
  ❌ 손상된 이미지: 0개
  ⚠️ 너무 작은 이미지: 0개
  🔄 잘못된 형식: 0개

🔍 어노테이션 무결성 검사 시작...
📋 어노테이션 무결성 검사 결과:
  ✅ 정상 어노테이션: 50개
  ❌ 손상된 JSON: 0개
  🔗 매칭되지 않는 쌍: 0개
  📐 잘못된 바운딩 박스: 0개

📊 데이터셋 통계 정보 생성...

📐 이미지 크기 통계:
  평균 크기: 976 x 1280
  최소 크기: 976 x 1280
  최대 크기: 976 x 1280
  평균 파일 크기: 1.73 MB

🏷️ 어노테이션 통계:
  총 객체 수: 50개
  이미지당 평균 객체 수: 1.0개
  고유 알약 종류: 14개
  가장 많은 알약 코드:
    3482: 14개
    1899: 9개
    3350: 7개
    3543: 6개
    2482: 5개

📋 품질 검사 보고서 생성 중...
✅ 보고서 저장 완료: ./data/prototype_data/quality_report.json

🎯 전체 품질 점수: 100.0%

🔍 발견된 문제들 (최대 5개씩 표시):

✅ 데이터 품질 검사 완료!


## 2단계 이미지 전처리

In [26]:
import os
import glob
import json
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
from collections import defaultdict
import shutil

### a. 크기 정규화 (1280x1280)

In [27]:
def calculate_resize_params(original_width, original_height, target_size=1280):
    """리사이즈 파라미터 계산 (Aspect Ratio 보존)"""
    # 긴 쪽을 기준으로 스케일 계산
    scale = min(target_size / original_width, target_size / original_height)

    # 새로운 크기 계산
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # 패딩 계산 (중앙 정렬)
    pad_left = (target_size - new_width) // 2
    pad_top = (target_size - new_height) // 2
    pad_right = target_size - new_width - pad_left
    pad_bottom = target_size - new_height - pad_top

    return {
        'scale': scale,
        'new_size': (new_width, new_height),
        'padding': (pad_left, pad_top, pad_right, pad_bottom),
        'target_size': (target_size, target_size)
    }

### b. Aspect Ratio 보존 및 패딩

In [28]:
def resize_image_with_padding(image, target_size=1280, fill_color=(0, 0, 0)):
    """이미지를 비율 유지하며 리사이즈하고 패딩 추가"""
    original_width, original_height = image.size
    params = calculate_resize_params(original_width, original_height, target_size)

    # 1. 비율 유지하며 리사이즈
    resized = image.resize(params['new_size'], Image.Resampling.LANCZOS)

    # 2. 새로운 캔버스 생성 (검은색 배경)
    new_image = Image.new('RGB', (target_size, target_size), fill_color)

    # 3. 중앙에 배치
    pad_left, pad_top = params['padding'][:2]
    new_image.paste(resized, (pad_left, pad_top))

    return new_image, params

### c. 밝기/대비 정규화

In [29]:
def analyze_image_brightness(image):
    """이미지 밝기 분석"""
    # Grayscale로 변환하여 밝기 계산
    gray = image.convert('L')
    pixels = np.array(gray)

    stats = {
        'mean_brightness': np.mean(pixels),
        'std_brightness': np.std(pixels),
        'min_brightness': np.min(pixels),
        'max_brightness': np.max(pixels),
        'histogram': np.histogram(pixels, bins=256, range=(0, 256))[0]
    }

    return stats

In [30]:
def normalize_brightness_contrast(image, target_mean=128, target_std=40):
    """밝기와 대비 정규화"""
    # 현재 밝기 분석
    stats = analyze_image_brightness(image)
    current_mean = stats['mean_brightness']
    current_std = stats['std_brightness']

    # 극단적인 경우 처리
    if current_std < 5:  # 너무 단조로운 이미지
        current_std = 20

    # 대비 조정
    contrast_factor = target_std / current_std
    contrast_factor = max(0.5, min(2.0, contrast_factor))  # 0.5~2.0 범위로 제한

    # 밝기 조정
    brightness_offset = target_mean - current_mean
    brightness_factor = 1.0 + (brightness_offset / 255.0)
    brightness_factor = max(0.3, min(1.7, brightness_factor))  # 0.3~1.7 범위로 제한

    # PIL ImageEnhance 사용
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)

    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)

    return image

### e. 색상 정규화

In [31]:
def analyze_color_distribution(image):
    """색상 분포 분석"""
    # RGB 채널별 분석
    pixels = np.array(image)

    stats = {
        'mean_rgb': np.mean(pixels, axis=(0, 1)),
        'std_rgb': np.std(pixels, axis=(0, 1)),
        'min_rgb': np.min(pixels, axis=(0, 1)),
        'max_rgb': np.max(pixels, axis=(0, 1))
    }

    return stats

In [32]:
def normalize_white_balance(image, reference_gray=128):
    """간단한 화이트 밸런스 조정"""
    pixels = np.array(image, dtype=np.float32)

    # 각 채널의 평균값 계산
    mean_rgb = np.mean(pixels, axis=(0, 1))

    # 조정 비율 계산 (그레이 기준으로)
    gray_target = reference_gray
    adjustments = gray_target / (mean_rgb + 1e-6)  # 0으로 나누기 방지

    # 극단적인 조정 방지
    adjustments = np.clip(adjustments, 0.7, 1.4)

    # 각 채널에 조정 적용
    adjusted_pixels = pixels * adjustments
    adjusted_pixels = np.clip(adjusted_pixels, 0, 255).astype(np.uint8)

    return Image.fromarray(adjusted_pixels)

In [41]:
def detect_background_color(image, sample_size=50):
    """배경색 추정 (이미지 가장자리 샘플링)"""
    pixels = np.array(image)
    height, width = pixels.shape[:2]

    # 가장자리 픽셀들 수집
    edge_pixels = []

    # 상단, 하단 가장자리
    edge_pixels.extend(pixels[0, :sample_size].reshape(-1, 3))
    edge_pixels.extend(pixels[-1, :sample_size].reshape(-1, 3))

    # 좌측, 우측 가장자리
    edge_pixels.extend(pixels[:sample_size, 0].reshape(-1, 3))
    edge_pixels.extend(pixels[:sample_size, -1].reshape(-1, 3))

    edge_pixels = np.array(edge_pixels)

    # 평균 색상 계산
    background_color = np.mean(edge_pixels, axis=0).astype(int)

    return tuple(background_color)

In [42]:
def standardize_background(image, target_bg_color=(240, 240, 240), threshold=30):
    """배경 표준화 (간단한 버전)"""
    # 현재 배경색 추정
    current_bg = detect_background_color(image)

    pixels = np.array(image)

    # 배경 픽셀 마스크 생성 (현재 배경색과 유사한 픽셀들)
    bg_mask = np.all(np.abs(pixels - current_bg) < threshold, axis=2)

    # 배경 픽셀을 목표 색상으로 변경
    result_pixels = pixels.copy()
    result_pixels[bg_mask] = target_bg_color

    return Image.fromarray(result_pixels)

### f. 데이터셋 전처리

In [43]:
def process_single_image(image_path, output_path, target_size=1280,
                        normalize_brightness=True, normalize_color=True,
                        standardize_bg=True):
    """단일 이미지 전처리"""
    try:
        # 이미지 로드
        with Image.open(image_path) as original_image:
            # RGB로 변환 (RGBA나 다른 모드인 경우)
            if original_image.mode != 'RGB':
                original_image = original_image.convert('RGB')

            # 원본 정보 저장
            original_size = original_image.size

            # 1. 크기 정규화
            processed_image, resize_params = resize_image_with_padding(
                original_image, target_size
            )

            # 2. 밝기/대비 정규화
            if normalize_brightness:
                processed_image = normalize_brightness_contrast(processed_image)

            # 3. 색상 정규화 (화이트 밸런스)
            if normalize_color:
                processed_image = normalize_white_balance(processed_image)

            # 4. 배경 표준화
            if standardize_bg:
                processed_image = standardize_background(processed_image)

            # 출력 디렉토리 생성
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # 저장
            processed_image.save(output_path, 'PNG', quality=95)

            return {
                'status': 'success',
                'original_size': original_size,
                'processed_size': (target_size, target_size),
                'resize_params': resize_params,
                'file_size_mb': os.path.getsize(output_path) / (1024*1024)
            }

    except Exception as e:
        return {
            'status': 'error',
            'error': str(e),
            'original_size': None,
            'processed_size': None
        }

In [44]:
def update_annotation_coordinates(annotation_path, output_annotation_path, resize_params):
    """리사이즈에 따른 어노테이션 좌표 업데이트"""
    try:
        with open(annotation_path, 'r', encoding='utf-8') as f:
            annotation_data = json.load(f)

        scale = resize_params['scale']
        pad_left, pad_top = resize_params['padding'][:2]

        # 어노테이션 좌표 업데이트
        if 'annotations' in annotation_data:
            for ann in annotation_data['annotations']:
                if 'bbox' in ann:
                    # 바운딩 박스 좌표 변환
                    x, y, w, h = ann['bbox']

                    # 스케일 적용
                    new_x = x * scale
                    new_y = y * scale
                    new_w = w * scale
                    new_h = h * scale

                    # 패딩 오프셋 적용
                    new_x += pad_left
                    new_y += pad_top

                    ann['bbox'] = [new_x, new_y, new_w, new_h]

        # 이미지 크기 정보 업데이트
        if 'images' in annotation_data:
            for img_info in annotation_data['images']:
                img_info['width'] = resize_params['target_size'][0]
                img_info['height'] = resize_params['target_size'][1]

        # 저장
        os.makedirs(os.path.dirname(output_annotation_path), exist_ok=True)
        with open(output_annotation_path, 'w', encoding='utf-8') as f:
            json.dump(annotation_data, f, indent=2, ensure_ascii=False)

        return True

    except Exception as e:
        print(f"어노테이션 업데이트 실패: {annotation_path}, 오류: {e}")
        return False

In [45]:
def preprocess_dataset(input_path, output_path, target_size=1280):
    """전체 데이터셋 전처리"""
    print(f"🚀 데이터셋 전처리 시작!")
    print(f"📁 입력: {input_path}")
    print(f"📁 출력: {output_path}")
    print(f"🎯 목표 크기: {target_size}x{target_size}")
    print("=" * 50)

    results = {
        'total_images': 0,
        'processed_images': 0,
        'failed_images': [],
        'total_annotations': 0,
        'processed_annotations': 0,
        'processing_stats': []
    }

    # 출력 디렉토리 구조 생성
    for subdir in ['train_images', 'test_images', 'train_annotations']:
        os.makedirs(os.path.join(output_path, subdir), exist_ok=True)

    # 1. 훈련 이미지 처리
    train_images = glob.glob(os.path.join(input_path, "train_images", "*.png"))
    results['total_images'] += len(train_images)

    print(f"🔄 훈련 이미지 처리 중... ({len(train_images)}개)")

    for i, img_path in enumerate(train_images):
        img_name = os.path.basename(img_path)
        output_img_path = os.path.join(output_path, "train_images", img_name)

        # 이미지 처리
        result = process_single_image(img_path, output_img_path, target_size)

        if result['status'] == 'success':
            results['processed_images'] += 1
            results['processing_stats'].append(result)

            # 해당하는 어노테이션 처리
            img_name_no_ext = img_name.replace('.png', '')
            annotation_files = glob.glob(
                os.path.join(input_path, "train_annotations", "*", "*", f"{img_name_no_ext}.json")
            )

            if annotation_files:
                annotation_path = annotation_files[0]
                # 상대 경로 계산
                rel_path = os.path.relpath(annotation_path,
                                         os.path.join(input_path, "train_annotations"))
                output_annotation_path = os.path.join(output_path, "train_annotations", rel_path)

                # 어노테이션 좌표 업데이트
                if update_annotation_coordinates(annotation_path, output_annotation_path,
                                               result['resize_params']):
                    results['processed_annotations'] += 1

                results['total_annotations'] += 1
        else:
            results['failed_images'].append({
                'path': img_path,
                'error': result['error']
            })

        # 진행률 출력
        if (i + 1) % 50 == 0:
            print(f"  진행률: {i + 1}/{len(train_images)}")

    # 2. 테스트 이미지 처리
    test_images = glob.glob(os.path.join(input_path, "test_images", "*.png"))
    results['total_images'] += len(test_images)

    print(f"\n🔄 테스트 이미지 처리 중... ({len(test_images)}개)")

    for i, img_path in enumerate(test_images):
        img_name = os.path.basename(img_path)
        output_img_path = os.path.join(output_path, "test_images", img_name)

        result = process_single_image(img_path, output_img_path, target_size)

        if result['status'] == 'success':
            results['processed_images'] += 1
            results['processing_stats'].append(result)
        else:
            results['failed_images'].append({
                'path': img_path,
                'error': result['error']
            })

        # 진행률 출력
        if (i + 1) % 50 == 0:
            print(f"  진행률: {i + 1}/{len(test_images)}")

    # 결과 요약
    print(f"\n📊 전처리 결과 요약:")
    print(f"  ✅ 처리된 이미지: {results['processed_images']}/{results['total_images']}")
    print(f"  ✅ 처리된 어노테이션: {results['processed_annotations']}/{results['total_annotations']}")
    print(f"  ❌ 실패한 이미지: {len(results['failed_images'])}개")

    if results['processing_stats']:
        avg_file_size = np.mean([stat['file_size_mb'] for stat in results['processing_stats']])
        print(f"  📏 평균 파일 크기: {avg_file_size:.2f} MB")

    # 실패한 이미지들 출력
    if results['failed_images']:
        print(f"\n❌ 실패한 이미지들:")
        for fail in results['failed_images'][:5]:  # 최대 5개만 출력
            print(f"  - {fail['path']}: {fail['error']}")
        if len(results['failed_images']) > 5:
            print(f"  ... 외 {len(results['failed_images']) - 5}개")

    print("\n✅ 데이터셋 전처리 완료!")
    return results

### 메인 실행 함수

In [46]:
def quick_preprocess_dev_data():
    """개발용 데이터 빠른 전처리"""
    return preprocess_dataset("./data/dev_data", "./data/dev_data/preprocessed_dev_data")

def quick_preprocess_prototype_data():
    """프로토타입용 데이터 빠른 전처리"""
    return preprocess_dataset("./data/prototype_data", "./data/prototype_data/preprocessed_prototype_data")

In [47]:
if __name__ == "__main__":
    print("📖 사용법:")
    print("quick_preprocess_prototype_data()  # 프로토타입 데이터 전처리")
    print("quick_preprocess_dev_data()        # 개발용 데이터 전처리")
    print("preprocess_dataset(input_path, output_path)  # 커스텀 전처리")

    # 실행 함수 선언
    quick_preprocess_prototype_data()

📖 사용법:
quick_preprocess_prototype_data()  # 프로토타입 데이터 전처리
quick_preprocess_dev_data()        # 개발용 데이터 전처리
preprocess_dataset(input_path, output_path)  # 커스텀 전처리
🚀 데이터셋 전처리 시작!
📁 입력: ./data/prototype_data
📁 출력: ./data/prototype_data/preprocessed_prototype_data
🎯 목표 크기: 1280x1280
🔄 훈련 이미지 처리 중... (50개)
  진행률: 50/50

🔄 테스트 이미지 처리 중... (25개)

📊 전처리 결과 요약:
  ✅ 처리된 이미지: 75/75
  ✅ 처리된 어노테이션: 50/50
  ❌ 실패한 이미지: 0개
  📏 평균 파일 크기: 1.57 MB

✅ 데이터셋 전처리 완료!


## 3단계: 어노테이션 전처리

In [53]:
import os
import json
import glob

### a. COCO JSON 읽기

In [86]:
def extract_pill_code_simple(filename):
    """파일명에서 알약 코드 추출 (간단 버전)"""
    try:
        # 확장자 제거
        name = filename.replace('.png', '').replace('.json', '')

        # '_' 전까지가 알약 코드
        if '_' in name:
            pill_code = name.split('_')[0]
        else:
            pill_code = name

        return pill_code
    except:
        return filename

### b. YOLO 포맷으로 변환

In [87]:
def simple_coco_to_yolo(data_path, output_dir):
    """간단한 COCO → YOLO 변환기"""
    print("🔄 간단한 COCO → YOLO 변환 시작...")

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # 1. 모든 JSON 파일 찾기
    json_files = glob.glob(os.path.join(data_path, "train_annotations", "*", "*", "*.json"))
    print(f"📋 발견된 JSON 파일: {len(json_files)}개")

    if not json_files:
        print("❌ JSON 파일을 찾을 수 없습니다!")
        return

    # 2. 모든 알약 코드 수집
    all_pill_codes = set()
    valid_files = []

    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # COCO 포맷 확인
            if 'images' in data and 'annotations' in data:
                for img_info in data['images']:
                    filename = img_info['file_name']
                    pill_code = extract_pill_code_simple(filename)
                    all_pill_codes.add(pill_code)

                valid_files.append((json_file, data))
        except Exception as e:
            print(f"⚠️ 파일 읽기 실패: {json_file} - {e}")

    print(f"✅ 유효한 파일: {len(valid_files)}개")
    print(f"🏷️ 발견된 알약 코드: {len(all_pill_codes)}개")

    # 3. 클래스 매핑 생성 (리스트 기반, 안전함)
    sorted_codes = sorted(list(all_pill_codes))
    code_to_id = {}

    for i, code in enumerate(sorted_codes):
        code_to_id[code] = i

    print(f"📊 클래스 매핑:")
    for i, code in enumerate(sorted_codes[:5]):
        print(f"  {i}: {code}")
    if len(sorted_codes) > 5:
        print(f"  ... 외 {len(sorted_codes)-5}개")

    # 4. 클래스 파일 생성
    classes_file = os.path.join(output_dir, 'classes.txt')
    with open(classes_file, 'w', encoding='utf-8') as f:
        for code in sorted_codes:
            f.write(f"{code}\n")
    print(f"✅ 클래스 파일 생성: {classes_file}")

    # 5. YOLO 라벨 파일 생성
    converted_count = 0

    for json_file, coco_data in valid_files:
        # 이미지별로 그룹화
        images_by_id = {img['id']: img for img in coco_data['images']}

        # 각 이미지에 대해 YOLO 파일 생성
        for img_info in coco_data['images']:
            img_id = img_info['id']
            filename = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']

            # 해당 이미지의 어노테이션 찾기
            image_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_id]

            if image_annotations:  # 어노테이션이 있는 경우만
                # YOLO 파일명 생성
                base_name = filename.replace('.png', '').replace('.jpg', '')
                yolo_file = os.path.join(output_dir, f"{base_name}.txt")

                # 알약 코드 추출
                pill_code = extract_pill_code_simple(filename)
                class_id = code_to_id.get(pill_code, 0)  # 기본값 0

                # YOLO 라벨 작성
                with open(yolo_file, 'w') as f:
                    for ann in image_annotations:
                        # COCO bbox: [x, y, width, height]
                        bbox = ann['bbox']
                        x, y, w, h = bbox

                        # YOLO 포맷으로 변환: [center_x, center_y, width, height] (상대 좌표)
                        center_x = (x + w/2) / img_width
                        center_y = (y + h/2) / img_height
                        width_rel = w / img_width
                        height_rel = h / img_height

                        # YOLO 라인 작성
                        f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width_rel:.6f} {height_rel:.6f}\n")

                converted_count += 1

    print(f"✅ 변환 완료: {converted_count}개 이미지")
    print(f"📁 출력 디렉토리: {output_dir}")

    # 6. 간단한 통계
    yolo_files = glob.glob(os.path.join(output_dir, "*.txt"))
    yolo_files = [f for f in yolo_files if not f.endswith('classes.txt')]  # classes.txt 제외

    print(f"\n📊 변환 결과:")
    print(f"  📄 생성된 YOLO 파일: {len(yolo_files)}개")
    print(f"  🏷️ 총 클래스 수: {len(sorted_codes)}개")

    return {
        'output_dir': output_dir,
        'classes_file': classes_file,
        'yolo_files_count': len(yolo_files),
        'total_classes': len(sorted_codes),
        'class_mapping': code_to_id
    }

### c. 클래스 매핑 생성

In [88]:
def quick_convert_prototype():
    """프로토타입 데이터 빠른 변환"""
    return simple_coco_to_yolo(
        "./data/prototype_data/preprocessed_prototype_data",
        "./data/prototype_datayolo_prototype"
    )

def quick_convert_dev():
    """개발용 데이터 빠른 변환"""
    return simple_coco_to_yolo(
        "./data/dev_data/preprocessed_dev_data",
        "./data/dev_data/yolo_dev"
    )

### 메인 실행부

In [89]:
def verify_yolo_output(output_dir):
    """YOLO 출력 결과 확인"""
    print(f"🔍 YOLO 출력 확인: {output_dir}")

    # 클래스 파일 확인
    classes_file = os.path.join(output_dir, 'classes.txt')
    if os.path.exists(classes_file):
        with open(classes_file, 'r') as f:
            classes = f.read().strip().split('\n')
        print(f"📋 클래스 수: {len(classes)}")
        print(f"   첫 5개: {classes[:5]}")

    # YOLO 라벨 파일 확인
    txt_files = [f for f in glob.glob(os.path.join(output_dir, "*.txt"))
                 if not f.endswith('classes.txt')]
    print(f"📄 YOLO 라벨 파일: {len(txt_files)}개")

    if txt_files:
        # 첫 번째 파일 내용 확인
        sample_file = txt_files[0]
        print(f"📝 샘플 파일: {os.path.basename(sample_file)}")
        with open(sample_file, 'r') as f:
            lines = f.readlines()[:3]  # 처음 3줄만
        for i, line in enumerate(lines):
            print(f"   {i+1}: {line.strip()}")

In [90]:
if __name__ == "__main__":
    print("📖 간단한 COCO → YOLO 변환기")
    print("사용법:")
    print("  quick_convert_prototype()  # 프로토타입 데이터 변환")
    print("  quick_convert_dev()        # 개발용 데이터 변환")
    print("  verify_yolo_output('./data/yolo_prototype')  # 결과 확인")

    prototype_result = quick_convert_prototype()

📖 간단한 COCO → YOLO 변환기
사용법:
  quick_convert_prototype()  # 프로토타입 데이터 변환
  quick_convert_dev()        # 개발용 데이터 변환
  verify_yolo_output('./data/yolo_prototype')  # 결과 확인
🔄 간단한 COCO → YOLO 변환 시작...
📋 발견된 JSON 파일: 50개
✅ 유효한 파일: 50개
🏷️ 발견된 알약 코드: 48개
📊 클래스 매핑:
  0: K-001900-016548-018110-033009
  1: K-001900-016548-019607-021026
  2: K-001900-016548-021026-021771
  3: K-001900-016548-021771-033009
  4: K-001900-016548-027926-044199
  ... 외 43개
✅ 클래스 파일 생성: ./data/prototype_datayolo_prototype/classes.txt
✅ 변환 완료: 50개 이미지
📁 출력 디렉토리: ./data/prototype_datayolo_prototype

📊 변환 결과:
  📄 생성된 YOLO 파일: 50개
  🏷️ 총 클래스 수: 48개


In [48]:
# 전역 설정 (권장) - 이 Colab 세션에서 계속 사용
!git config --global user.name "Dongjin-1203"
!git config --global user.email "hambur1203@gmail.com"

In [49]:
!git add .

In [50]:
!git commit -m "데이터 파이프라인 코드 수정: 이미지 전처리"

[main f028191] 데이터 파이프라인 코드 수정: 이미지 전처리
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite data_preprocess/Untitled0.ipynb (98%)


In [51]:
!git pull origin main

From https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project
 * branch            main       -> FETCH_HEAD
Already up to date.


In [52]:
!git push origin main

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 6.51 KiB | 350.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project.git
   d59ff28..f028191  main -> main
